In [1]:
se_f = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/enrichment/annotation_clones/SE.rds"
outdir = "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/enrichment/annotation_clones/development/DE_large"
n_donors = 4
# se_f = "/data2/mito_lineage/output/annotation_clones/data/cd34norm/MTblacklist/SE.rds"
# outdir = "/data2/mito_lineage/output/annotation_clones/data/cd34norm/MTblacklist/DE_large"

# If more than 2 clones pass the cdf_thresh, use that, o.w use n_top_clones
cdf_thresh = 0.5
n_top_clones = 3

min_pct = 0 #0.01
logfc_threshold = 0

genome = "/data2/mito_lineage/data/external/GRCh38_MT_blacklist/fasta/genome.fa"
cores=16
utils_script <- "/data2/mito_lineage/R_scripts/seurat_utils.R"

In [5]:
de_out <- file.path(outdir, paste0( "minPct", min_pct, "logFCThresh", logfc_threshold))
de_out

[1] "/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/enrichment/annotation_clones/development/DE_large/minPct0logFCThresh0"

In [8]:
dir.create(de_out, recursive=T)

Warning message in dir.create(de_out, recursive = T):
“'/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/enrichment/annotation_clones/development/DE_large/minPct0logFCThresh0' already exists”


In [9]:
source(utils_script)

Attaching SeuratObject



In [10]:
library(Seurat)
library(Signac)
library(ggplot2)
library(patchwork)
set.seed(1234)
library(magrittr)
library(cowplot)
library(metap)
library(dplyr)
library(future)
plan("multiprocess", workers = cores)
#options(future.globals.maxSize = 50000 * 1024^2) # for 50 Gb RAM
options(future.globals.maxSize = 8000 * 1024^2)
library("ComplexHeatmap")


Attaching package: ‘cowplot’


The following object is masked from ‘package:patchwork’:

    align_plots



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“Strategy 'multiprocess' is deprecated in future (>= 1.20.0). Instead, explicitly specify either 'multisession' or 'multicore'. In the current R session, 'multiprocess' equals 'multicore'.”
Loading required package: grid

ComplexHeatmap version 2.9.4
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export stat

## Import seurat object

In [11]:
if (file.exists(file.path(outdir, "se.chromvar.rds"))){
    print("Loading se with chromvar")
    se <- readRDS(file=file.path(outdir, "se.chromvar.rds"))    
}else{
    print("Running chromvar")
    se = readRDS(se_f)
#     source("/data2/mito_lineage/R_scripts/addPWM.R")
    se <- get.pwm(se, genome=genome)
    saveRDS(object=se, file=file.path(outdir, "se.chromvar.rds"))    
}



[1] "Running chromvar"




Loading required package: BSgenome

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, whi

In [12]:
se[[]]

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,⋯,Row.names,index,raw.ID,new.index,condition,donor,donor_index,lineage,lineage_index,name
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<I<chr>>,<dbl>,<chr>,<dbl>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<fct>
Control_AAACGAAAGAGGTCCA-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAAAGAGGTCCA-1,0,AAACGAAAGAGGTCCA-1,1,Control,3,1,19,1,3_19
Control_AAACGAAAGCGATACG-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAAAGCGATACG-1,1,AAACGAAAGCGATACG-1,2,Control,1,1,10,1,1_10
Control_AAACGAAAGTCGTGAG-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAAAGTCGTGAG-1,2,AAACGAAAGTCGTGAG-1,3,Control,0,1,12,1,0_12
Control_AAACGAACAATAGTGA-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAACAATAGTGA-1,3,AAACGAACAATAGTGA-1,4,Control,3,2,None,NA,None
Control_AAACGAACACAATAAG-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAACACAATAAG-1,4,AAACGAACACAATAAG-1,5,Control,0,2,18,1,0_18
Control_AAACGAACACTGATAC-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAACACTGATAC-1,5,AAACGAACACTGATAC-1,6,Control,2,1,None,NA,None
Control_AAACGAACAGTAACCT-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAACAGTAACCT-1,6,AAACGAACAGTAACCT-1,7,Control,2,2,4,1,2_4
Control_AAACGAACATCGTGAT-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAACATCGTGAT-1,7,AAACGAACATCGTGAT-1,8,Control,3,3,None,NA,None
Control_AAACGAAGTAACGGTG-1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,⋯,Control_AAACGAAGTAACGGTG-1,8,AAACGAAGTAACGGTG-1,9,Control,3,4,None,NA,None


In [13]:
counts <- se[[]] %>%
          group_by(condition, donor, lineage, name) %>%
          summarize(size=n()) %>% filter(!name=="None")
counts

`summarise()` has grouped output by 'condition', 'donor', 'lineage'. You can override using the `.groups` argument.



condition,donor,lineage,name,size
<fct>,<fct>,<fct>,<fct>,<int>
Control,0,0,0_0,13
Control,0,1,0_1,95
Control,0,2,0_2,22
Control,0,3,0_3,2
Control,0,4,0_4,2
Control,0,5,0_5,2
Control,0,6,0_6,58
Control,0,7,0_7,26
Control,0,8,0_8,1


In [14]:
counts.norm <- counts %>% group_by(condition, donor) %>% mutate(total=sum(size)) %>% ungroup() %>% mutate(norm=size/total)

counts.norm 

condition,donor,lineage,name,size,total,norm
<fct>,<fct>,<fct>,<fct>,<int>,<int>,<dbl>
Control,0,0,0_0,13,606,0.021452145
Control,0,1,0_1,95,606,0.156765677
Control,0,2,0_2,22,606,0.036303630
Control,0,3,0_3,2,606,0.003300330
Control,0,4,0_4,2,606,0.003300330
Control,0,5,0_5,2,606,0.003300330
Control,0,6,0_6,58,606,0.095709571
Control,0,7,0_7,26,606,0.042904290
Control,0,8,0_8,1,606,0.001650165


In [15]:
counts.norm.sizes = counts.norm[order(counts.norm[,"norm"], decreasing=T),]
counts.norm.sizes

Warning message in xtfrm.data.frame(x):
“cannot xtfrm data frames”


condition,donor,lineage,name,size,total,norm
<fct>,<fct>,<fct>,<fct>,<int>,<int>,<dbl>
Flt3l,2,8,2_8,219,889,0.24634421
Control,2,8,2_8,163,727,0.22420908
Flt3l,2,7,2_7,156,889,0.17547807
Flt3l,1,10,1_10,212,1231,0.17221771
Control,2,7,2_7,115,727,0.15818432
Control,0,1,0_1,95,606,0.15676568
Control,1,10,1_10,93,598,0.15551839
Control,0,12,0_12,88,606,0.14521452
Control,2,17,2_17,103,727,0.14167813


In [16]:
clone.sizes <- counts.norm.sizes %>% 
               group_by(donor,lineage) %>% 
               summarize(norm.total=sum(norm), total=sum(size)) %>% 
               arrange(desc(norm.total)) %>% 
               mutate(name=factor(paste0(donor,"_", lineage)))


clone.sizes

`summarise()` has grouped output by 'donor'. You can override using the `.groups` argument.



donor,lineage,norm.total,total,name
<fct>,<fct>,<dbl>,<int>,<fct>
2,8,0.4705533,382,2_8
2,7,0.3336624,271,2_7
1,10,0.3277361,305,1_10
0,1,0.2959562,236,0_1
2,17,0.2642878,212,2_17
1,7,0.2570659,236,1_7
0,12,0.2567644,201,0_12
2,4,0.2232792,180,2_4
3,13,0.2120257,165,3_13


In [17]:
clone.sizes <- clone.sizes %>% group_by(donor) %>% arrange(donor, desc(norm.total)) %>% 
                               mutate(cdf.norm=cumsum(norm.total)/2, cdf =cumsum(total),
                                      index=1:n()) %>% 
                               ungroup %>% arrange(donor, desc(norm.total)) 
#head(clone.sizes,100)

In [18]:
large.clones <- clone.sizes %>% filter(cdf.norm<cdf_thresh)
large.clones <- check.if.one.clone(large.clones, clone.sizes, n_top_clones)
#large.clones$name = factor(paste0(large.clones$donor,"_", large.clones$lineage))
write.csv(large.clones, 
          file=file.path(outdir, "large_clones.csv"), quote=F)

large.clones

[1] 4
[1] "i 1 donor 1"
[1] "i 2 donor 2"
[1] "i 3 donor 3"
[1] "only 1 clone. Using top 3 clones"
[1] "i 4 donor 4"


donor,lineage,norm.total,total,name,cdf.norm,cdf,index
<fct>,<fct>,<dbl>,<int>,<fct>,<dbl>,<int>,<int>
0,1,0.2959562,236,0_1,0.1479781,236,1
0,12,0.2567644,201,0_12,0.2763603,437,2
0,10,0.2026478,167,0_10,0.3776842,604,3
0,19,0.1980654,159,0_19,0.4767169,763,4
1,10,0.3277361,305,1_10,0.1638681,305,1
1,7,0.2570659,236,1_7,0.2924010,541,2
1,0,0.1714882,155,1_0,0.3781451,696,3
1,13,0.1540486,142,1_13,0.4551694,838,4
2,8,0.4705533,382,2_8,0.2352766,382,1


## TF DE

## Filter se for cells in the large clones

In [19]:
se <- subset(se, subset= name %in% factor(large.clones$name))
se

An object of class Seurat 
170811 features across 2712 samples within 4 assays 
Active assay: chromvarnames (633 features, 0 variable features)
 3 other assays present: ATAC, RNA, chromvar
 2 dimensional reductions calculated: integrated_lsi, umap

## Run DE

In [23]:
for (d in 0:(n_donors-1)){
    print('donor')
    print(d)
    curr.outdir <- file.path(outdir, paste0("donor", d, "_TF"))
    dir.create(curr.outdir)
    donor.large.clones <- large.clones %>% filter(donor==d)
    clones.filt.ids <- sort(donor.large.clones$lineage)
    print('clones')
    print(clones.filt.ids)
    print(length(clones.filt.ids))
    se.filt <- subset(se, subset = (donor==d) & (lineage %in% donor.large.clones$lineage))
    Idents(se.filt) <- se.filt$lineage  
    print('se.filt')
    print(dim(se.filt[[]]))
    DefaultAssay(se.filt) <- "chromvarnames"
    pairs = combn(clones.filt.ids,2)
    for (i in 1:dim(combn(clones.filt.ids,2))[2]){
        print(pairs[,i])
        try({
            print(paste("clones", i))
            a = pairs[1,i]
            b = pairs[2,i]
            da <- FindMarkers(
              object = se.filt,
              ident.1 = a,
              ident.2 = b,
              only.pos = FALSE,
              mean.fxn = rowMeans,
              logfc.threshold = logfc_threshold,
              min.pct = min_pct, 
              #test.use = 'LR', latent.vars = 'peak_region_fragments' 
              fc.name = "avg_diff")
            curr.tf.da$p_val_adj_BH <- stats::p.adjust(curr.tf.da$p_val, method = "BH", n = length(curr.tf.da$p_val))
            write.csv(da, 
                      file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.csv")))
            gally <- GGally::ggpairs(da[,c("p_val", "p_val_adj")], aes(alpha = 0.4))
            ggsave(plot=gally, file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.pvalHist.png")))
            DefaultAssay(se.filt) <- "ATAC"
            raw.ids <- names(Motifs(se.filt)@motif.names)
            rownames(curr.tf.da) <- sapply((curr.tf.da$X), function(x){raw.ids[match(x, Motifs(se.filt)@motif.names)]})
            m <- MotifPlot(
              object = se.filt,
              motifs = head(rownames(curr.tf.da)),
              assay = 'ATAC'
            )
            ggsave(plot=m, 
                       file=file.path(curr.outdir, paste0("clones_",a,"__", b,".DE.TF.motif.top.png")))

            

            #plotDE(se, da, c, clust_outdir)

    })
        }
    }

[1] "donor"
[1] 0


Warning message in dir.create(curr.outdir):
“'/data2/mito_lineage/output/pipeline/cd34norm/MTblacklist/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_mgatkdonor/vireo/nclones20/enrichment/annotation_clones/development/DE_large/donor0_TF' already exists”


[1] "clones"
[1] 1  10 12 19
Levels: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 None
[1] 4
[1] "se.filt"
[1] 763  44
[1] 1  10
Levels: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 None
[1] "clones 1"


Saving 6.67 x 6.67 in image

Warning message in cor(x, y):
“the standard deviation is zero”


[1] 1  12
Levels: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 None
[1] "clones 2"


...

Saving 6.67 x 6.67 in image



[1] 8  19
Levels: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 None
[1] "clones 2"


Saving 6.67 x 6.67 in image

Warning message in cor(x, y):
“the standard deviation is zero”


[1] 13 19
Levels: 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 None
[1] "clones 3"


Saving 6.67 x 6.67 in image

Warning message in cor(x, y):
“the standard deviation is zero”


In [22]:
help(ggsave)

Warning message in seq_len(head.end.idx):
“first element used of 'length.out' argument”
ERROR while rich displaying an object: Error in seq_len(head.end.idx): argument must be coercible to non-negative integer

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     if (!mime %in% names(repr::mime2repr)) 
 .         stop("No repr_* for mimetype ", mime, " in repr::mime2repr")
 .     rpr <- repr::mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_conte